<a href="https://colab.research.google.com/github/2001harsh-mehra/vehicle-image-classification/blob/main/1_CNN_without_data_augmentation_Vehicle_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
vehicle_base_drive = '/content/drive/MyDrive/Vehicle Classification'
vehicle_zip_dir = os_veh.path.join(vehicle_base_drive, 'vehicles.zip')
vehicle_extract_dir = '/content/vehicle_data'

with vehiclezip.ZipFile(vehicle_zip_dir, 'r') as zip_ref:
    zip_ref.extractall(vehicle_extract_dir)
print("Unzipping done. Vehicle images are now ready!")

vehicle_main_dir = os_veh.path.join(vehicle_extract_dir, 'Vehicles')

for clas_vehicle in os_veh.listdir(vehicle_main_dir):
    clas_path_veh = os_veh.path.join(vehicle_main_dir, clas_vehicle)
    if os_veh.path.isdir(clas_path_veh):
        pictures = os_veh.listdir(clas_path_veh)
        full_veh_cnt = len(pictures)
        print(f"Vehicle Class: {clas_vehicle} — Total number of images: {full_veh_cnt}")

Unzipping done. Vehicle images are now ready!
Vehicle Class: Cars — Total number of images: 790
Vehicle Class: Ships — Total number of images: 800
Vehicle Class: Auto Rickshaws — Total number of images: 800
Vehicle Class: Planes — Total number of images: 800
Vehicle Class: Bikes — Total number of images: 800
Vehicle Class: Motorcycles — Total number of images: 800
Vehicle Class: Trains — Total number of images: 800
